# Immporting data

In [58]:
# Importing main modules
import pandas as pd
import numpy as np

In [59]:
# Importing paths
path1 = "../Data/To_analyze/Results_software/SIDER2_chr1/RUNS/run_20.csv"
path2 = "../Data/To_analyze/Results_software/SIDER2_chr32/RUNS/run_20.csv"

In [60]:
# Importing data
data_1 = pd.read_csv(path1, sep=",", header=None)
data_32 = pd.read_csv(path2, sep=",", header=None)

In [61]:
# Editing data to necessary columns
data_1 = data_1[[1, 3, 10, 11, 14, 15]]; data_1.columns = range(data_1.columns.size)
data_32 = data_32[[1, 3, 10, 11, 14, 15]]; data_32.columns = range(data_32.columns.size)

# Analyzing data

## Number of elements per chromosome and strand

In [62]:
# Group data by chromosome and strand
data_1_grouped = data_1.groupby([0, 4]).size()
data_32_grouped = data_32.groupby([0, 4]).size()

# Editing columns so that they are in a readable format
data_1_grouped = data_1_grouped.unstack(level=-1)
data_32_grouped = data_32_grouped.unstack(level=-1)

In [63]:
# Sizes of data:
print(data_1_grouped.shape, data_32_grouped.shape, sep="\n")

(36, 2)
(36, 2)


In [64]:
# Compare number of elements difference in each chromosome and strand minus
minus_diff = data_32_grouped.iloc[:, 0] - data_1_grouped.iloc[:, 0]


In [65]:
# Now, let's make it in percent
minus_diff_perc = round(((data_32_grouped.iloc[:, 0] - data_1_grouped.iloc[:, 0]) \
/ data_1_grouped.iloc[:, 0]) \
* 100, 2)

In [66]:
# Now with "plus" strand
plus_diff = data_32_grouped.iloc[:, 1] - data_1_grouped.iloc[:, 1]

In [67]:
# Now, let's make it in percent
plus_diff_perc = round(((data_32_grouped.iloc[:, 1] - data_1_grouped.iloc[:, 1]) \
/ data_1_grouped.iloc[:, 1]) \
* 100, 2)

In [68]:
diff_df = pd.DataFrame({"Minus_diff": minus_diff, "Minus_diff_perc": minus_diff_perc, "Plus_diff": plus_diff, "Plus_diff_perc": plus_diff_perc})

## Compare coordinates elements

In [102]:
data_1_subset1 = data_1[data_1[0] == "LinJ.01"]
data_32_subset1 = data_32[data_32[0] == "LinJ.01"]

In [88]:
test_1 = data_1.groupby([0, 4])
test_32 = data_32.groupby([0, 4])

In [89]:
type(test_1)

pandas.core.groupby.generic.DataFrameGroupBy

In [161]:
results = []

for index, ((group_name_1, group_data_1), (group_name_32, group_data_32)) in enumerate(zip(test_1, test_32), start=1):
    print("="*100)
    print(f"First group ID: {group_name_1}, with a shape of {group_data_1.shape}")
    print(f"Second group ID: {group_name_32}, with a shape of {group_data_32.shape}")

    # ================================================================================================================
    # Let's serach for exact values
    # ================================================================================================================
    # Now let's find the values in the first group that are in the second group:
    plus_values_matches = group_data_1.iloc[:,2].isin(group_data_32.iloc[:,2])
    exact_matches_start_pct = round((plus_values_matches.sum() / group_data_1.shape[0])*100, 2)
    print(f"In the Start coordinates there are exactly {plus_values_matches.sum()} matches out of {group_data_1.shape[0]} ==> {exact_matches_start_pct} % coincidence")

    minus_values_matches = group_data_1.iloc[:,3].isin(group_data_32.iloc[:,3])
    exact_matches_end_pct = round((minus_values_matches.sum() / group_data_1.shape[0])*100, 2)
    print(f"In the End coordinates there are exactly {minus_values_matches.sum()} matches out of {group_data_1.shape[0]} ==> {exact_matches_end_pct} % coincidence")

    # ================================================================================================================
    # Now, let's search for values in a range of 100 nt
    # ================================================================================================================
    tolerance = 200
    matches_start = pd.Series([False] * group_data_1.shape[0]) # boolean series about the data_1 shape
    matches_end = pd.Series([False] * group_data_1.shape[0]) # boolean series about the data_1 shape

    for i, (index1, row1) in enumerate(group_data_1.iterrows()):
        for _, row2 in group_data_32.iterrows():
            # Check for match in the 'start' columns
            if abs(row1[2] - row2[2]) <= tolerance:
                matches_start.iloc[i] = True
                # Continue checking for 'end' even after finding a 'start' match
            
            # Check for match in the 'end' columns
            if abs(row1[3] - row2[3]) <= tolerance:
                matches_end.iloc[i] = True
                
            # Once a match is found for both, you can break out of the loop
            if matches_start.iloc[i] and matches_end.iloc[i]:
                break
    
    tolerance_matches_start_pct = round((matches_start.sum() / group_data_1.shape[0])*100, 2)
    tolerance_matches_end_pct = round((matches_end.sum() / group_data_1.shape[0])*100, 2)

    print(f"\nWith TOLERANCE:")
    print(f"In the Start coordinates there are exactly {matches_start.sum()} matches out of {group_data_1.shape[0]} ==> {tolerance_matches_start_pct} % coincidence")
    print(f"In the End coordinates there are exactly {matches_end.sum()} matches out of {group_data_1.shape[0]} ==> {tolerance_matches_end_pct} % coincidence")

    # if index == 1: break
    # results.append([group_name_1[0],
    #                 group_name_1[1], 
    #                 exact_matches_start_pct, 
    #                 exact_matches_end_pct, 
    #                 tolerance_matches_start_pct, 
    #                 tolerance_matches_end_pct])

    
    results.append({'Group Name': group_name_1[0],
                    'Strand': group_name_1[1], 
                    'Exact Matches Start (%)': exact_matches_start_pct, 
                    'Exact Matches End (%)': exact_matches_end_pct, 
                    'Matches within Tolerance Start (%)': tolerance_matches_start_pct, 
                    'Matches within Tolerance End (%)': tolerance_matches_end_pct})

results_df = pd.DataFrame(results)

First group ID: ('LinJ.01', 'minus'), with a shape of (12, 6)
Second group ID: ('LinJ.01', 'minus'), with a shape of (14, 6)
In the Start coordinates there are exactly 3 matches out of 12 ==> 25.0 % coincidence
In the End coordinates there are exactly 10 matches out of 12 ==> 83.33 % coincidence

With TOLERANCE:
In the Start coordinates there are exactly 10 matches out of 12 ==> 83.33 % coincidence
In the End coordinates there are exactly 10 matches out of 12 ==> 83.33 % coincidence
First group ID: ('LinJ.01', 'plus'), with a shape of (21, 6)
Second group ID: ('LinJ.01', 'plus'), with a shape of (22, 6)
In the Start coordinates there are exactly 17 matches out of 21 ==> 80.95 % coincidence
In the End coordinates there are exactly 12 matches out of 21 ==> 57.14 % coincidence

With TOLERANCE:
In the Start coordinates there are exactly 19 matches out of 21 ==> 90.48 % coincidence
In the End coordinates there are exactly 20 matches out of 21 ==> 95.24 % coincidence
First group ID: ('LinJ.0